# [04] 모델 만들기2 - AuxConv

`AuxConv`는 `BaseConv`에서 추출된 mid(18x18), end(9x9) featuremap 중에서 end featuremap을 받아와, 여러 스케일의 featuremap으로 학습하는 모듈입니다. 우리는 mid, end와 그에 대해 AuxConv를 수행한 4개의 추가적인 feature map들(9x9, 5x5, 3x3, 1x1)을 활용하여 `PredConv`에서 객체 검출(Object Detection)을 수행할 것입니다.

---------------
`AuxConv` 는 여러 층의 Conv Layer에 featuremap을 통과시키면서 다양한 scale의 featuremap을 얻습니다. 아래는 VGG-Net을 기반으로 만든 SSD 검출기의 auxiliary conv층을 표현하고 있습니다. 아래의 이미지에서는 19x19의 feature map을 사용하고 있습니다.

- [참고] SSD(Single Shot Multibox Detector) 논문 : https://arxiv.org/pdf/1512.02325.pdf

In [2]:
from IPython.display import HTML, display

# Image from https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection
display(HTML("<img src='img/[04]auxconv.png'>"))

-------------------
## [Task 1] `AuxConv`로 featuremap 얻기

`AuxConv`에서는 18x18의 feature map을 받아 conv layer에 통과시킴으로써 9x9, 5x5, 3x3, 1x1의 feature map들을 얻습니다. 

### ToDo : AuxConv 연습하기

먼저, 아래의 `auxconv_practice`함수를 완성하여 5x5의 feature map을 얻어봅시다. 필요한 구성은 다음과 같습니다.

- `conv1` : 1x1 convolution (out_channels=256, stride=1, padding=0)
- activation : relu
- `conv2` : 3x3 convolution (out_channels=512, stride=2, padding=1)
- activation : relu

9x9의 feature map을 위의 layer들에 통과시킴으로써, 5x5의 feature map을 얻을 수 있습니다.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def auxconv_practice(featuremap):
    """featuremap을 받아 auxiliary conv를 통과시킵니다."""
    
    # [ToDo]: conv1을 구성합니다.
    conv1 = nn.??????(1280, ????, kernel_size=???, stride=???, padding=???)
    
    # [ToDo]: conv2를 구성합니다.
    conv2 = nn.Conv2d(????, ????, kernel_size=???, stride=???, padding=???)
    act = nn.ReLU()
    
    output = act(conv1(featuremap))
    output = act(conv2(output))
    
    return output

sample = torch.randn(4, 1280, 18, 18)
output = auxconv_practice(sample)

print('결과 feature map의 shape: {}'.format(output.shape))


결과 feature map의 shape: torch.Size([4, 512, 9, 9])


------------
## [Task 2] AuxConvluition Class

`AuxConv`도 이전 실습의 `BaseConv`와 마찬가지로 `nn.Module` Class로 만들어보도록 하겠습니다. 각 결과 feature map은 1x1 conv와 3x3 conv를 각각 한 번씩 거칠 때마다 얻게 됩니다.

### ToDo: nn.Module Class로 통합하기

`__init__()` 함수를 완성합니다. 각 layer의 구성은 다음과 같습니다.
- conva_1 : 1x1, (out_channels=256, stride=1, padding=0)
- conva_2 : 3x3, (out_channels=512, stride=2, padding=1)
- convb_1 : 1x1, (out_channels=128, stride=1, padding=0)
- convb_2 : 3x3, (out_channels=256, stride=2, padding=1)
- convc_1 : 1x1, (out_channels=128, stride=1, padding=0)
- convc_2 : 3x3, (out_channels=256, stride=1, padding=0)
- convd_1 : 1x1, (out_channels=128, stride=1, padding=0)
- convd_2 : 3x3, (out_channels=256, stride=1, padding=0)

`forward` 함수를 완성합니다.
- 2개의 conv층을 거칠 때마다 feature map 하나를 얻습니다.
- 각 conv층의 뒤에 relu activation 연산을 합니다.
- 9x9, 5x5, 3x3, 1x1의 feature map을 리턴합니다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AuxConvolution(nn.Module):
    
    # [ToDo]: __init__ 함수를 완성합니다. use_bias로 bias 사용 여부를 선택 가능하도록 합니다.
    def __init__(self, ????????):
        super(AuxConvolution, self).__init__()
        self.conv_a1 = nn.Conv2d(1280, 256, kernel_size=1, stride=1, padding=0, bias=??????)
        self.conv_a2 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=??????)
        
        self.conv_b1 = nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0, bias=??????)
        self.conv_b2 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=??????)
        
        self.conv_c1 = nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=??????)
        self.conv_c2 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0, bias=??????)
        
        self.conv_d1 = nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=??????)
        self.conv_d2 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0, bias=??????)
        
        self.init__conv2d(??????)
        
        
    def init__conv2d(self, ??????):
        """weight와 bias를 initialize합니다."""
        for c in self.children():
            if isinstance(c, nn.Conv2d):
                nn.init.xavier_uniform_(c.weight)
                if ??????:
                    nn.init.constant_(c.bias, 0.)
        
    
    # [ToDo]: forward 함수를 완성합니다.
    def forward(self, end_features):
        """
        end featuremap을 받아 conv layers를 통과시켜 9x9, 5x5, 3x3, 1x1의 feature map을 얻습니다.
        """
        out = ???????????????????
        out = ???????????????????
        features_a = out
        
        out = ???????????????????
        out = ???????????????????
        features_b = out
        
        out = ???????????????????
        out = ???????????????????
        features_c = out
        
        out = ???????????????????
        out = ???????????????????
        features_d = out
        
        return features_a, features_b, features_c, features_d


Class를 완성했다면, BaseConvolution과 연동하여 AuxConvolution을 거친 featuremap의 결과값을 확인해 봅시다.

In [5]:
from materials.det_modules import BaseConvolution

baseconv = BaseConvolution()

sample = torch.randn(4, 3, 300, 300)
mid, end = baseconv(sample)

print('\nmid shape {}'.format(mid.shape))
print('end shape {}'.format(end.shape))

Loaded pretrained weights for efficientnet-b0

mid shape torch.Size([4, 112, 18, 18])
end shape torch.Size([4, 1280, 9, 9])


In [6]:
# AuxConvolution을 테스트합니다.

# [ToDo]: AuxConvolution을 생성합니다.
auxconv = ???????????????????

# [ToDo]: end feature를 통과시켜 4개의 featuremap을 얻습니다.
??????????????????? = ???????????????????

print('\nmap a shape : {}'.format(features_a.shape))
print('map b shape : {}'.format(features_b.shape))
print('map c shape : {}'.format(features_c.shape))
print('map d shape : {}'.format(features_d.shape))


map a shape : torch.Size([4, 512, 9, 9])
map b shape : torch.Size([4, 256, 5, 5])
map c shape : torch.Size([4, 256, 3, 3])
map d shape : torch.Size([4, 256, 1, 1])


---------
### <생각해 봅시다>

- 1x1 conv와 3x3 conv가 하는 역할은 무엇인가요?
- 왜 다양한 크기의 feature map을 구성하고자 하나요?

------------